# Kernel Methoden

In dieser Übung werden wir und verschiedene Kernel Methoden ansehen und diese vergleichen.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split



Wir verwenden das *Red Wine Quality* Datenset und wollen ein Modell trainieren, welches die Qualität des Weines anhand verschiedener Parameter vorhersagen kann. Zuerst laden wir das Datenset als *pandas DataFrame*:

In [ ]:

df = pd.read_csv("/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")
data_train, data_test = train_test_split(df.copy(), test_size=0.2, random_state=42)
data_train.head()


Die Qualität ist ein Integer Wert zwischen 3 und 8. Wir können sowohl Klassifikations- als auch Regressionsmodelle verwenden, da es eine Skala ist.

In [ ]:
print("Min Quality:", df.quality.min())
print("Max Quality:", df.quality.max())

Wir können die Verteilung der Qualität in einem Histogram plotten und sehen, dass ein Großteil der Weine eine Qualität $\leq 6$ besitzen.


In [ ]:
df.quality.hist()

## 1a) Kernel Methoden Vergleich

Der folgende Code verwendet eine *Kernel Ridge Regression* um die Qualität vorherzusagen. 


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, make_scorer

X_train, y_train = data_train.drop(["quality"], axis=1), data_train.quality
X_test, y_test = data_test.drop(["quality"], axis=1), data_test.quality

# In sklearn ist ein höherer Score immer besser. der mean_absolute_error ist aber am besten, wenn er klein ist.
# Wenn wir den Scorer erstellen nehmen wir also -mean_absoute_error als Bewertungsmaß. Dazu setzten wir greater_is_better=False.
# Dementsprechend werden die Scores im Grid Search auch negativ sein und der Score, der am nähesten zu 0 ist der beste.
score = make_scorer(mean_absolute_error, greater_is_better=False)  

# make_scorer erstellt im Prinzip die folgende Funktion
# def score(model, X_true, y_true)
#     y_pred = model.predict(X_true)
#     return - mean_absolute_error(y_true, y_pred)

pipeline = Pipeline([
    ("scale", StandardScaler()),
    ("ridge", KernelRidge(kernel="rbf"))
])

model = GridSearchCV(pipeline, [{"ridge__alpha": [0.001, 0.01, 0.1, 1], "ridge__gamma": [0.001, 0.01, 0.03, 0.05, 0.1], "ridge__kernel": ["rbf"]}], 
                     n_jobs=-1, scoring=score, cv = 5)
model.fit(X_train, y_train)
pd.DataFrame(model.cv_results_).sort_values("rank_test_score").head()

- Trainiere eine *Support Vektor Regression* und einen *Support Vektor Klassifikator* mit einem *`rbf` Kernel*. Benutze einen StandardScaler als Vorbereitungsschritt und wähle die Hyperparameter mit `GridSearchCV`. Wir wollen das Modell anhand der *Mean Absolute Errors* bewerten. Setzte dazu `scoring=score` in `GridSearchCV`. Welches Modell ist das beste?

---
Um den Grid Search schneller zu machen, kannst du `n_jobs=-1` setzen. Dadurch werden die verschiedenen Modelle und Splits auf unterschiedliche Prozesse verteilt und parallel ausgeführt.

---


In [ ]:
from sklearn.svm import SVR, SVC
# SVR

In [ ]:
# SVC

- Welchen Fehler (*Mean Absolute Error*) erzielt das beste Modell auf der Testmenge?

## 1b) Binäre Support Vektor Klassifikation
Ein Gourmet-Restaurant will Wein einkaufen und möglichst keinen Wein mit schlechter Qualität in der Karte haben. Dazu will es das *Red Wine Quality* Datenset verwenden um einen Klassifikator zu trainieren, welcher guten Wein von schlechten unterscheiden kann. Guter Wein mit einer Qualität $\gt 6$ bekommt das Label $1$ und schlechter Wein mit Qualität $\leq 6$ bekommt das Label $0$. Für das Restaurant ist ein hoher *Precision Score* sehr wichtig. Das bedeutet, wenn der Klassifikator einen Wein als gut einstuft, dann soll dieser Wein mit hoher "Wahrscheinlichkeit" tatsächlich gut sein. Der *Recall Score*, die "Wahrscheinlichkeit", dass ein guter Wein als gut erkannt wird, ist eher nebensächlich. Mit anderen Worten, das Restaurant kann es eher verkraften einen guten Wein nicht anzubieten, als dass es dem Kunden einen schlechten Wein serviert.


Der folgende Code erstellt dieses neue *Target*. Beachte, dass die Bezeichung jetzt `X_train_clf` und `y_train_clf` ist.

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, make_scorer

df_clf = df.copy()

df_clf["goodquality"] = (df_clf.quality > 6).astype(int)
df_clf = df_clf.drop("quality", axis=1)
data_train_clf, data_test_clf = train_test_split(df_clf, test_size=0.2, random_state=43, stratify=df_clf.goodquality)


X_train_clf, y_train_clf = data_train_clf.drop(["goodquality"], axis=1), data_train_clf.goodquality
X_test_clf, y_test_clf = data_test_clf.drop(["goodquality"], axis=1), data_test_clf.goodquality

Wir können sehen, dass das Datenset jetzt sehr unausgewogen ist und die positive Klasse eher selten ist.

In [ ]:
y_train_clf.value_counts()

- Trainiere ein `SVC` Modell. Benutze wieder `StandardScaler`, `GridSearchCV` und `StratifiedKFold` (setze dazu `cv=StratifiedKFold()` in `GridSearchCV`).  Benutze den `f1_score` und setzte `probability=True` damit die `SVC` Wahrscheinlichkeiten berechnen kann.

In [ ]:
scorer = make_scorer(f1_score)


- Berechne die Wahrheitsmatrix des Trainingssets und des besten Modells. Benutze dazu `cross_val_predict`, benutze wieder `cv=StratifiedKFold()`.
- Berechne *Recall* und *Precision Score* anhand des Trainingssets.

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, recall_score, precision_score


- Plotte die Precision und den Recall gegen den Schwellenwert (Threshold; in $[0,1]$), welcher die Klassenzugehörigkeit festlegt. Wir verwenden `cross_val_predict` und setzen `method='predict_proba'` um die Klassenzugehörigkeitswahrscheinlichkeiten auszugeben.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_curve, roc_curve

def plot_precision_recall(model):
    y_pred = cross_val_predict(model.best_estimator_, X_train_clf, y_train_clf, method='predict_proba', cv=StratifiedKFold(5))
    
    # wir nehmen die Klassenzugehörigkeitswahrscheinlichkeit der positiven Klasse
    y_pred = y_pred[:, 1] 
    
    precision, recall, thresholds = precision_recall_curve(y_train_clf, y_pred)

    plt.figure()
    plt.plot(thresholds, precision[:-1], "b--", label="Precision")
    plt.plot(thresholds, recall[:-1], "g-", label="Recall")
    plt.xlabel('Threshold')
    plt.legend()
    plt.grid()
    plt.show()


- Plotte die ROC-Curve.

In [ ]:
from sklearn.metrics import roc_curve

- Das Restaurant will ein Modell, welches eine *Precision* von mindestens 80% hat. Welchen Schwellenwert (*Threshold*) würdest du wählen?
- Wie hoch ist Precision und Recall auf dem Testset mit und ohne Berücksichtigung des Schwellenwerts?

finale Version